<a href="https://colab.research.google.com/github/jasondupree/jasondupree.github.io/blob/main/simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DATA CREATION

In [21]:
import numpy as np
import pandas as pd
import random
import string
from datetime import datetime, timedelta

# Set random seed for reproducibility
np.random.seed(42)

# ✅ Global Parameters for Full Control
NUM_WEEKS = 52
NUM_CHANNELS = 3
NUM_SOLD_TO_NUMBERS = 6
NUM_PRODUCTS = 10
NUM_SIZES = 3

# ✅ Channels - Configurable
CHANNELS = ["NIKE DIRECT DIGITAL", "NIKE DIRECT STORES", "WHOLESALE"][:NUM_CHANNELS]

# ✅ Channel Selling Prices
SELLING_PRICE_BY_CHANNEL = {
    "NIKE DIRECT DIGITAL": 50,
    "NIKE DIRECT STORES": 45,
    "WHOLESALE": 40
}

# ✅ Liquidation Selling Price
LIQUIDATION_SELLING_PRICE = 20

# ✅ End Product Offer Date (EPOD)
EPOD_BASE_DATE = "2024-12-31"

# ✅ Starting Inventory by Season
STARTING_INVENTORY_BY_SEASON = {
    "Summer": 80,
    "Fall": 60,
    "Winter": 40,
    "Spring": 50
}

# ✅ Starting Contract Quantities per Channel per Season
STARTING_CONTRACT_QTY_BY_SEASON_CHANNEL = {
    "Summer": {"NIKE DIRECT DIGITAL": 50, "NIKE DIRECT STORES": 0, "WHOLESALE": 20},
    "Fall": {"NIKE DIRECT DIGITAL": 40, "NIKE DIRECT STORES": 25, "WHOLESALE": 15},
    "Winter": {"NIKE DIRECT DIGITAL": 30, "NIKE DIRECT STORES": 20, "WHOLESALE": 10},
    "Spring": {"NIKE DIRECT DIGITAL": 45, "NIKE DIRECT STORES": 25, "WHOLESALE": 15},
}

# ✅ Assign Season based on Week
def get_season(week):
    if 1 <= week <= 13:
        return "Summer"
    elif 14 <= week <= 26:
        return "Fall"
    elif 27 <= week <= 39:
        return "Winter"
    else:
        return "Spring"

# ✅ Generate unique Product_Cds
def generate_product_cds():
    return [f"{''.join(random.choices(string.ascii_uppercase + string.digits, k=6))}-{''.join(random.choices(string.digits, k=3))}" for _ in range(NUM_PRODUCTS)]

# ✅ Generate unique Size_Cds
def generate_size_cds():
    return [str(size) for size in range(8, 8 + NUM_SIZES)]

# ✅ Assign EPOD dynamically (20% to 2024-12-31, 80% later)
def assign_epod_to_products(product_cds):
    num_epod_products = max(1, int(len(product_cds) * 0.2))
    epod_assignment = {}

    epod_products = random.sample(product_cds, num_epod_products)
    for product in product_cds:
        if product in epod_products:
            epod_assignment[product] = EPOD_BASE_DATE
        else:
            epod_assignment[product] = (datetime.strptime(EPOD_BASE_DATE, "%Y-%m-%d") + timedelta(days=365)).strftime("%Y-%m-%d")
    return epod_assignment

# ✅ Generate unique Sold-To-Numbers
def generate_sold_to_numbers():
    sold_to_numbers = {}
    for channel in CHANNELS:
        num_sold_to = NUM_SOLD_TO_NUMBERS // len(CHANNELS)
        sold_to_numbers[channel] = [
            f"000000{np.random.randint(1000, 9999)}" if channel.startswith("NIKE DIRECT") else f"{np.random.randint(1, 9)}{np.random.randint(100000000, 999999999)}"
            for _ in range(num_sold_to)
        ]
    return sold_to_numbers

# ✅ Generate arriving inventory
def generate_arriving_inventory():
    return {week: np.random.choice([0, np.random.randint(20, 50)], p=[0.5, 0.5]) for week in range(1, NUM_WEEKS + 1)}

# SIMULATION

In [22]:
# ✅ Function to determine selling price based on EPOD
def determine_selling_price(channel, product_cd, epod_dict, week):
    epod_date = datetime.strptime(epod_dict[product_cd], "%Y-%m-%d")
    current_date = datetime.strptime(EPOD_BASE_DATE, "%Y-%m-%d") + timedelta(weeks=week)

    return LIQUIDATION_SELLING_PRICE if current_date > epod_date else SELLING_PRICE_BY_CHANNEL[channel]

# ✅ Function to process demand fulfillment
def process_demand(week, channel, product_cd, size_cd, sold_to, arriving_inventory, contract_inventory_dict):
    demand = np.random.randint(5, 20)

    key = (channel, product_cd, size_cd, get_season(week))

    if key not in contract_inventory_dict:
        contract_inventory_dict[key] = STARTING_CONTRACT_QTY_BY_SEASON_CHANNEL[get_season(week)][channel]

    starting_contract_qty = contract_inventory_dict[key]
    contract_call_off_qty = min(demand, starting_contract_qty)
    remaining_demand = demand - contract_call_off_qty
    ending_contract_qty = starting_contract_qty - contract_call_off_qty

    captured_demand = min(remaining_demand, arriving_inventory)
    lost_demand = remaining_demand - captured_demand
    arriving_inventory -= captured_demand

    return {
        "Contract Call-Off Qty": contract_call_off_qty,
        "Ending Contract Qty": ending_contract_qty,
        "Captured Demand": contract_call_off_qty + captured_demand,
        "Lost Demand": lost_demand,
        "Remaining Inventory": arriving_inventory,
    }

# ✅ Master Simulation Function
def run_simulation(product_cds, size_cds, sold_to_numbers, arriving_inventory_dict, epod_dict):
    cumulative_holding_cost = 0
    holding_cost_per_unit = 2
    cumulative_captured_demand = 0
    cumulative_lost_demand = 0
    contract_inventory_dict = {}

    data = []

    for week in range(1, NUM_WEEKS + 1):
        season = get_season(week)
        arriving_inventory = arriving_inventory_dict[week]
        is_first_row = True

        for product_cd in product_cds:
            for size_cd in size_cds:
                for channel, sold_to_list in sold_to_numbers.items():
                    for sold_to in sold_to_list:

                        # Process Demand
                        demand_data = process_demand(week, channel, product_cd, size_cd, sold_to, arriving_inventory, contract_inventory_dict)

                        # Determine Selling Price
                        selling_price = determine_selling_price(channel, product_cd, epod_dict, week)

                        # Compute Holding Cost
                        weekly_holding_cost = (demand_data["Remaining Inventory"] + demand_data["Ending Contract Qty"]) * holding_cost_per_unit
                        cumulative_holding_cost += weekly_holding_cost

                        # Update cumulative demand tracking
                        cumulative_captured_demand += demand_data["Captured Demand"]
                        cumulative_lost_demand += demand_data["Lost Demand"]

                        # Append row data
                        data.append({
                            "Week": week,
                            "Season": season,
                            "Sold-To-Number": sold_to,
                            "Channel": channel.upper(),
                            "Product_Cd": product_cd,
                            "Size_Cd": size_cd,
                            "Contract Owner": channel if demand_data["Ending Contract Qty"] > 0 else None,
                            "Starting Contract Qty": contract_inventory_dict[(channel, product_cd, size_cd, season)],
                            **demand_data,
                            "Arriving Inventory": arriving_inventory if is_first_row else 0,
                            "Selling Price": selling_price,
                            "EPOD": epod_dict[product_cd],
                            "Cumulative Captured Demand": cumulative_captured_demand,
                            "Cumulative Lost Demand": cumulative_lost_demand,
                            "Weekly Holding Cost": weekly_holding_cost,
                            "Cumulative Holding Cost": cumulative_holding_cost
                        })

                        is_first_row = False

    return pd.DataFrame(data)

# EXECUTION

In [24]:
product_cds = generate_product_cds()
size_cds = generate_size_cds()
sold_to_numbers = generate_sold_to_numbers()
arriving_inventory_dict = generate_arriving_inventory()
epod_dict = assign_epod_to_products(product_cds)

df = run_simulation(product_cds, size_cds, sold_to_numbers, arriving_inventory_dict, epod_dict)

from IPython.display import display
display(df)

# ✅ Install xlsxwriter if not installed
try:
    import xlsxwriter
except ModuleNotFoundError:
    !pip install xlsxwriter
    import xlsxwriter

# ✅ Export to Excel
output_filename = "Simulation.xlsx"

# Create an Excel writer object
with pd.ExcelWriter(output_filename, engine="xlsxwriter") as writer:
    df.to_excel(writer, sheet_name="current", index=False)  # Save simulation results in 'current' tab
    pd.DataFrame().to_excel(writer, sheet_name="proposal", index=False)  # Blank 'proposal' tab

print(f"✅ Simulation results saved to: {output_filename}")

# ✅ Print Final Cumulative Metrics
final_cumulative_captured_demand = df["Cumulative Captured Demand"].iloc[-1]
final_cumulative_lost_demand = df["Cumulative Lost Demand"].iloc[-1]
final_cumulative_holding_cost = df["Cumulative Holding Cost"].iloc[-1]

print("\n📊 **Final Simulation Metrics**")
print(f"🔹 Final Cumulative Captured Demand: {final_cumulative_captured_demand}")
print(f"🔸 Final Cumulative Lost Demand: {final_cumulative_lost_demand}")
print(f"💰 Final Cumulative Holding Cost: ${final_cumulative_holding_cost}")

,Week,Season,Sold-To-Number,Channel,Product_Cd,Size_Cd,Contract Owner,Starting Contract Qty,Contract Call-Off Qty,Ending Contract Qty,Captured Demand,Lost Demand,Remaining Inventory,Arriving Inventory,Selling Price,EPOD,Cumulative Captured Demand,Cumulative Lost Demand,Weekly Holding Cost,Cumulative Holding Cost
0,1,Summer,0000002680,NIKE DIRECT DIGITAL,0FNUFO-779,8,NIKE DIRECT DIGITAL,50,10,40,10,0,0,0,50,2025-12-31,10,0,80,80
1,1,Summer,0000009433,NIKE DIRECT DIGITAL,0FNUFO-779,8,NIKE DIRECT DIGITAL,50,5,45,5,0,0,0,50,2025-12-31,15,0,90,170
2,1,Summer,0000006682,NIKE DIRECT STORES,0FNUFO-779,8,None,0,0,0,0,8,0,0,45,2025-12-31,15,8,0,170
3,1,Summer,0000009084,NIKE DIRECT STORES,0FNUFO-779,8,None,0,0,0,0,15,0,0,45,2025-12-31,15,23,0,170
4,1,Summer,8871375173,WHOLESALE,0FNUFO-779,8,WHOLESALE,20,16,4,16,0,0,0,40,2025-12-31,31,23,8,178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9355,52,Spring,0000009433,NIKE DIRECT DIGITAL,56A63V-914,10,NIKE DIRECT DIGITAL,45,8,37,8,0,0,0,50,2025-12-31,104716,7623,74,563088
9356,52,Spring,0000006682,NIKE DIRECT STORES,56A63V-914,10,NIKE DIRECT STORES,25,19,6,19,0,0,0,45,2025-12-31,104735,7623,12,563100
9357,52,Spring,0000009084,NIKE DIRECT STORES,56A63V-914,10,NIKE DIRECT STORES,25,5,20,5,0,0,0,45,2025-12-31,104740,7623,40,563140
9358,52,Spring,8871375173,WHOLESALE,56A63V-914,10,WHOLESALE,15,6,9,6,0,0,0,40,2025-12-31,104746,7623,18,563158


✅ Simulation results saved to: Simulation.xlsx

📊 **Final Simulation Metrics**
🔹 Final Cumulative Captured Demand: 104758
🔸 Final Cumulative Lost Demand: 7623
💰 Final Cumulative Holding Cost: $563164
